# LINEAR FEEDBACK SHIFT REGISTERS

## NOTEBOOK 1 - PRELIMINARIES

In these notebooks we demonstrate key methods and functionality in the `lfsr` library. Modules are found in the directory `lfsr/`. The present notebook concerns generating and plotting linear feedback shift registers (LFSRs).

See [Wikipedia: Linear feedback shift registers](https://en.wikipedia.org/wiki/Linear-feedback_shift_register) for background details on LFSRs.

### LIBRARY IMPORT

Import the `LFSR` class object from `lfsr`.

In [1]:
from lfsr_library import LFSR

### INITIALIZE THE LFSR

The `LFSR` object above is initialised by two keyword arguments:

    - the degree, of type int
    - tap positions, of type list[int]

**Note.** The position numbering for binary sequences used here is LSb. So in the 4-bit number `1000 = 1 << 3`, the bit `1` is in position `3`.


#### ERRONEOUS INIT

When initialising taps, ensure that the position is less than the degree, otherwise `LFSR` will raise an error and fail to initialise as demonstrated below.

In [2]:
degree_test: int = 5
tap_positions_test: list[int] = [2, 4, 7]

lfsr_test = LFSR(degree=degree_test, tap_positions=tap_positions_test)

IndexError: Invalid tap positions. Positioning is 0-indexed.

#### VALID INIT

A valid LFSR-initialisation requires passing tap positions which are strictly less than the degree. We can then print the feedback polynomial by calling `.feedback_polynomial` to see an algebraic representation of our LFSR.

In [3]:
deg: int = 5
taps: list[int] = [0, 3, 4]

lfsr = LFSR(degree=deg, tap_positions=taps)

### FIRST PROPERTIES

On initialisation, the degree, tap positions and polynomial representation can be seen through the following calls.

In [4]:
print(f'the degree is: {lfsr.degree}')
print(f'the tap positions are: {lfsr.tap_positions}')
print(f'the feedback polynomial is: {lfsr.feedback_polynomial}')

the degree is: 5
the tap positions are: [0, 3, 4]
the feedback polynomial is: X^0 + X^3 + X^4 + X^5


With the `sympy` library, the feedback polynomial can be represented more cleanly. 

Call `.feedback_polynomial_sp` on `LFSR` as follows.

In [5]:
lfsr.feedback_polynomial_sp

x**5 + x**4 + x**3 + 1

### GENERATE

#### LFSR MECHANICS

An LFSR takes a sequence of bits, termed a *bit state* and, on each iteration, returns

    - an output bit
    - a new bit state

The degree of the LFSR determines the length of the bit state, i.e., number of bits in the state.

If $|B\rangle$ denotes a bit state, the least significant bit (LSb) in $|B\rangle$ is the output bit. This is the right-most bit in $|B\rangle$. The new state $|B^\prime\rangle$ is then determined as follows. The tap positions in the LFSR single out a collection of bits in $|B\rangle$. These are then XOR'd (i.e., added as elements in $\mathbb Z_2$) and appended to the *most* significant bit position (MSb) in $|B\rangle$. This is the left-most bit. After then dropping the LSb, we arrive at the new bit state $|B^\prime\rangle$. 

**Illustration.** Consider $|B\rangle = 10110$. This can be transformed by an LFSR of degree `5`. Consider tap positions at `3` and `4`. See that $\mathrm{LSb}(|B\rangle) = 0$. This is the first output of the LFSR. To construct the new bit state $|B^\prime\rangle$, firstly drop this LSb. The bits at positions `3` and `4` in $|B\rangle$ are `1` and `0`. XORing these gives `1` which is now appended to the MSb position, yielding the new bit state 

\begin{align}
|B^\prime\rangle = 11011
\end{align}

The string of output bits after $N$ many iterations is referred to as the LFSR's *bitstream*, or, *stream*.

#### STATE AND STREAM GENERATION

The generate method `generate()` can be called on our lfsr object to generate a log of bit state transitions and a bitstream. It passes arguments

    - a seed, being an appropriately sized bit state, either as a string or binary string
    - an integer number of times to iterate

The bitstream and log of bit states transitioned through can be retrievd through attribute calls `.stream` and `.log` respectively. See below for results of our lfsr object on the bit state $|B\rangle  = 10110$ from above.

In [6]:
SEED: int = 0b10110
ITERATIONS: int = 15

lfsr.generate(bitseq=SEED, iterations=ITERATIONS)
stream: str = lfsr.stream
log: list[str] = lfsr.log

print(f'the bitstream is: {stream}')
for bitstate in log:
    print(bitstate)

the bitstream is: 0110111100100001
10110
11011
11101
11110
01111
00111
10011
01001
00100
00010
00001
10000
11000
01100
10110
11011


### LFSR PERIOD

Every LFSR has a period. This is defined to be the number of iterations until the bit state returned coincides with the seed bit state. For an LFSR of a given degree $d$, its *maximal* possible period is $2^{d-1}$. Sub-maximal LFSRs are those with period less than the maximal. 

After `.generate()` is called on the lfsr object, its period can be retrieved through `.period` as follows.


In [13]:
print(lfsr.period)

14


Since `14 != 2**5-1 = 31`, this LFSR is *sub-maximal*. 

**Note.** While the seed bit state $|B\rangle$ can aid in calculating the period of an LFSR, the property of it being maximal or sub-maximal will not depend on the seed state.

Accordingly, it is in principle possible to determine if any LFSR of degree $d$ is maximal. This is by simply starting with *any* bit state and iterating $2^d-1$-many times, stopping if a period is found beforehand. Of course, for large enough degree $d$, this brute force approach is computationally infeasible.